２番に実行するコード

役割：データの調整

データダウンロード先：https://covid19-osaka.info/

In [34]:
import numpy as np
import csv
import pandas as pd
import copy

In [35]:
#start_day:開始日
#end_day：終了日
from datetime import datetime as dt
date_type = '%Y-%m-%d'

In [36]:
#inputのデータがheaderで何番目にいるのかを確認
def get_num(input_item, header, num):
    for i in input_item:
        count=0
        for h in header:
            if i == h:
                num.append(count)
            count += 1

In [37]:
input_data_filename = 'summary.csv'
bed_data_filename = 'bed_data.csv'
before_filename = 'osaka_data_before.csv'
input_items = ['日付', '検査件数','陽性人数', '現在陽性者数', '退院', '死亡', 'リンク不明者'] #入力に使いたい変数
bed_rate = ['病床使用率'] #bed関係で使いたいデータ

In [38]:
#開始日と終了日を参照する
df = pd.read_csv('./data/'+bed_data_filename, encoding='shift-jis', index_col=0, parse_dates=['日付'])
start_day = df.index[0]
end_day = df.index[-1]
start_day

Timestamp('2020-07-30 00:00:00')

In [39]:
with open('./data/'+input_data_filename, encoding='shift-jis', newline='') as f:
    csvreader = csv.reader(f) 
    header = next(csvreader)# 見出し行は別扱い
    
    #input_itemと同じ要素のみを抽出
    col_num = []
    get_num(input_items,header,col_num)
    '''for i in input_items:
        count=0
        for h in header:
            if i == h:
                col_num.append(count)
            count += 1'''
    
    raw_data=[]        
    for row in csvreader:
        if dt.strptime(row[0], date_type)>=start_day and dt.strptime(row[0], date_type)<=end_day:
            content=[]
            for i in col_num:
                content.append(row[i])
            raw_data.append(content)

with open('./data/'+bed_data_filename, encoding='shift-jis', newline='') as f:
    csvreader = csv.reader(f) 
    header = next(csvreader)# 見出し行は別扱い
    
    rate_num = []
    get_num(bed_rate,header,rate_num)
    
    for row in csvreader:
        for r in raw_data:
            if dt.strptime(r[0], date_type)==dt.strptime(row[0], date_type):
                r.append(row[rate_num[0]])

for i in bed_rate:
    input_items.append(i) #使いたいデータに'病床'を追加

In [40]:
#順番などを変更した新たなデータを形成
with open('./data/'+before_filename, 'w', newline="") as f:
    writer = csv.writer(f)
    writer.writerow(input_items)
    for i in raw_data:
        writer.writerow(i)

    
with open('./data/'+before_filename) as f:
    print(f.read())

日付,検査件数,陽性人数,現在陽性者数,退院,死亡,リンク不明者,病床使用率
2020-07-30,2070,190,1154,121,0,135,21.0
2020-07-31,2328,216,1265,104,1,146,23.0
2020-08-01,2293,195,1342,118,0,137,24.0
2020-08-02,1562,194,1445,91,0,142,29.0
2020-08-03,1266,81,1423,102,1,55,30.0
2020-08-04,2379,193,1509,106,1,129,30.0
2020-08-05,2107,196,1538,167,0,122,29.0
2020-08-06,2481,225,1504,258,1,147,31.0
2020-08-07,2558,255,1660,97,2,143,31.0
2020-08-08,1961,178,1667,171,0,130,34.0
2020-08-09,1730,195,1754,107,1,119,36.0
2020-08-10,1009,123,1704,173,0,59,38.0
2020-08-11,1682,101,1641,164,1,42,38.0
2020-08-12,2686,184,1674,150,1,111,39.0
2020-08-13,2428,177,1719,129,3,109,39.0
2020-08-14,2505,192,1722,184,5,94,39.0
2020-08-15,2048,151,1700,173,0,88,42.0
2020-08-16,1747,147,1692,157,0,84,44.0
2020-08-17,823,71,1633,130,5,41,43.0
2020-08-18,2599,185,1700,105,6,96,45.0
2020-08-19,2923,187,1718,164,5,105,46.0
2020-08-20,2247,132,1731,123,1,68,46.0
2020-08-21,2500,166,1710,177,5,102,46.0
2020-08-22,2066,134,1751,92,1,92,45.0
2020-08-23,1817,1

In [41]:
input_items

['日付', '検査件数', '陽性人数', '現在陽性者数', '退院', '死亡', 'リンク不明者', '病床使用率']

In [42]:
#加工後データの準備
#加工前の項目取得
additional_items = copy.deepcopy(input_items)
#削除項目
drop_items = ['リンク不明者']

after_raw_data=[] 
beds = []
hospitals = []

In [43]:
df = pd.read_csv('./data/'+bed_data_filename, index_col=[0], encoding='shift-jis', parse_dates=True)
beds_header = df.columns.values

#要素の追加
#特定の文字列を含むものを入力項目に使う
for i in beds_header:
    if ('入院' in i):
        additional_items.append(i)
        hospitals.append(i)
    elif ('病床確保数' in i):
        additional_items.append(i)
        beds.append(i)

#要素の削除
for i in drop_items:
    if i in additional_items:
        additional_items.remove(i)

In [44]:
beds

['重症病床確保数', '軽症中等症病床確保数']

In [45]:
hospitals

['重症入院患者数', '軽症中等症入院患者数']

In [46]:
input_items

['日付', '検査件数', '陽性人数', '現在陽性者数', '退院', '死亡', 'リンク不明者', '病床使用率']

In [47]:
additional_items

['日付',
 '検査件数',
 '陽性人数',
 '現在陽性者数',
 '退院',
 '死亡',
 '病床使用率',
 '重症病床確保数',
 '重症入院患者数',
 '軽症中等症病床確保数',
 '軽症中等症入院患者数']

In [48]:
with open('./data/'+before_filename, encoding='shift-jis', newline='') as f:
    csvreader = csv.reader(f) 
    header = next(csvreader)
    
    col_num = []
    get_num(additional_items, header, col_num)
            
    for row in csvreader:
        if dt.strptime(row[0], date_type)>=start_day and dt.strptime(row[0], date_type)<=end_day:
            content=[]
            for i in col_num:
                content.append(row[i])
            after_raw_data.append(content)
            
with open('./data/'+bed_data_filename, encoding='shift-jis', newline='') as f:
    csvreader = csv.reader(f) 
    header = next(csvreader)# 見出し行は別扱い
    
    #病床使用率は加工前にもあるので、今回追加するものだけ指定
    col_num = []
    get_num(beds,header,col_num)
    get_num(hospitals,header,col_num)
    
    for row in csvreader:
        for r in after_raw_data:
            if dt.strptime(r[0], date_type)==dt.strptime(row[0], date_type):
                for i in col_num:
                        if i != 0:
                            r.append(row[i])
            
'''with open('./data/'+bed_data_filename, encoding='shift-jis', newline='') as f:
    csvreader = csv.reader(f) 
    header = next(csvreader)# 見出し行は別扱い

    #additional_inputと同じ要素のみを抽出
    col_num = []
    beds_num = []
    hos_num = []
    get_num(beds, header, beds_num)
    get_num(hospitals, header, hos_num)
    get_num(additional_items, header, col_num)
         
    for row in csvreader:
        #print(row[0])
        if dt.strptime(row[0], '%Y-%m-%d')>=start_day and dt.strptime(row[0], '%Y-%m-%d')<=end_day:
            content=[]
            for r in after_raw_data:
                #print(r[0])
                if dt.strptime(r[0], '%Y-%m-%d')==dt.strptime(row[0], '%Y-%m-%d'):
                    for i in col_num:
                        if i != 0:
                            r.append(row[i])
                    #病床占有率の追加
                    total_beds=0
                    for b in beds_num:
                        total_beds+=float(row[b])
                    total_hospitals=0
                    for h in hos_num:
                        total_hospitals+=float(row[h])
                    r.append(total_hospitals/total_beds*100)
    
       
items = input_items
for i in additional_items:
    if i != '日付':
        items.append(i)
items.append('病床占有率')


print(items)
print(beds_num)
print(hos_num)
print(raw_data)'''

"with open('./data/'+bed_data_filename, encoding='shift-jis', newline='') as f:\n    csvreader = csv.reader(f) \n    header = next(csvreader)# 見出し行は別扱い\n\n    #additional_inputと同じ要素のみを抽出\n    col_num = []\n    beds_num = []\n    hos_num = []\n    get_num(beds, header, beds_num)\n    get_num(hospitals, header, hos_num)\n    get_num(additional_items, header, col_num)\n         \n    for row in csvreader:\n        #print(row[0])\n        if dt.strptime(row[0], '%Y-%m-%d')>=start_day and dt.strptime(row[0], '%Y-%m-%d')<=end_day:\n            content=[]\n            for r in after_raw_data:\n                #print(r[0])\n                if dt.strptime(r[0], '%Y-%m-%d')==dt.strptime(row[0], '%Y-%m-%d'):\n                    for i in col_num:\n                        if i != 0:\n                            r.append(row[i])\n                    #病床占有率の追加\n                    total_beds=0\n                    for b in beds_num:\n                        total_beds+=float(row[b])\n               

In [49]:
with open('./data/osaka_data_after.csv', 'w', newline="") as f:
    writer = csv.writer(f)
    writer.writerow(additional_items)
    for i in after_raw_data:
        writer.writerow(i)

    
with open('./data/osaka_data_after.csv') as f:
    print(f.read())

日付,検査件数,陽性人数,現在陽性者数,退院,死亡,病床使用率,重症病床確保数,重症入院患者数,軽症中等症病床確保数,軽症中等症入院患者数
2020-07-30,2070,190,1154,121,0,21.0,188,1069,18,245
2020-07-31,2328,216,1265,104,1,23.0,188,1069,19,275
2020-08-01,2293,195,1342,118,0,24.0,188,1069,20,288
2020-08-02,1562,194,1445,91,0,29.0,188,1069,23,341
2020-08-03,1266,81,1423,102,1,30.0,188,1069,27,346
2020-08-04,2379,193,1509,106,1,30.0,188,1069,26,346
2020-08-05,2107,196,1538,167,0,29.0,188,1069,30,334
2020-08-06,2481,225,1504,258,1,31.0,188,1069,35,359
2020-08-07,2558,255,1660,97,2,31.0,188,1069,36,353
2020-08-08,1961,178,1667,171,0,34.0,188,1069,40,387
2020-08-09,1730,195,1754,107,1,36.0,188,1069,38,411
2020-08-10,1009,123,1704,173,0,38.0,188,1069,41,436
2020-08-11,1682,101,1641,164,1,38.0,188,1069,49,424
2020-08-12,2686,184,1674,150,1,39.0,188,1069,55,430
2020-08-13,2428,177,1719,129,3,39.0,188,1069,53,437
2020-08-14,2505,192,1722,184,5,39.0,188,1069,64,428
2020-08-15,2048,151,1700,173,0,42.0,188,1069,70,461
2020-08-16,1747,147,1692,157,0,44.0,188,1069,72,4